In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
pd.__version__


In [ ]:
spark.conf.set("fs.azure.account.key.stupramonitoreomercado.dfs.core.windows.net", "xxx")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
#spark.conf.set("spark.sql.shuffle.partitions",3000)
#spark.conf.set("spark.default.parallelism",3000)

In [ ]:
df2 = spark.read.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/intervinientes_sin_duplicados.parquet")
#display(from_json(df2.select("col"))
display(df2)

In [ ]:
df2.count()

In [ ]:
#df2 = df2.withColumn("id", F.monotonically_increasing_id())

df2 = df2.withColumn("id", 
		   F.row_number().over(Window.orderBy("LLAVE_COMP"))
     )

df2.show()

In [ ]:
df2.createOrReplaceTempView("intervinientes")

In [ ]:
# listado de funciones   https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

df_clean =   spark.sql("""
SELECT  *,
trim( regexp_replace( regexp_replace(  regexp_replace( upper(PERSONA),  ' +', ' ') ,  ',+', ','    ) , '[^a-zA-Z ,]+' , '' ) )  as PERSONA_CLEAN  , 
trim(NUMERO_DOCUMENTO) as NUMERO_DOCUMENTO_CLEAN 
FROM intervinientes
""")
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DEL ', ' '))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE LA ', ' '))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE LAS ', ' '))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE ', ' '))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' LOS ', ' '))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DEL ', ''))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE LA ', ''))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE LAS ', ''))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE ', ''))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^LOS ', ''))
df_clean = df_clean.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^LA ', ''))
df_clean = df_clean.withColumn('NUMERO_DOCUMENTO_CLEAN', F.regexp_replace('NUMERO_DOCUMENTO_CLEAN', r'^-', ''))
df_clean = df_clean.withColumn('TIPO_DOCUMENTO', F.trim('TIPO_DOCUMENTO'))

def count_chars(nombre):
  if nombre is None :
    return 0
  else:
    return len(nombre)

count_chars_udf = udf(count_chars, IntegerType() )

def count_words(nombre):
  return len(nombre.split(" ") )

count_words_udf = udf(count_words, IntegerType() )

def count_commas(nombre):
  return len(nombre.split(",") )

count_commas_udf = udf(count_commas, IntegerType() )

def classify_type(tipo_documento):
  if ( tipo_documento == "E" or tipo_documento == "P" ):
    return "EXTRANJERO"
  else:
    return "NACIONAL"

classify_types_udf = udf(classify_type, StringType() )

#  Notas: hay algunas personas que separan apellido y nombre usando una coma 
df_clean = df_clean.withColumn("NUMERO_DOCUMENTO_CLEAN_LEN", count_chars_udf(df_clean.NUMERO_DOCUMENTO_CLEAN)  )
df_clean = df_clean.withColumn("persona_num_words", count_words_udf(df_clean.PERSONA_CLEAN)  )
df_clean = df_clean.withColumn("persona_num_commas", count_commas_udf(df_clean.PERSONA_CLEAN)  )
df_clean = df_clean.withColumn("TIPO", classify_types_udf(df_clean.TIPO_DOCUMENTO)  )
#df_clean.show(truncate=False) 

display(df_clean)
#df_clean.show(truncate=False)

In [ ]:
profile = ProfileReport(df_clean.select("*").toPandas(), title="Pandas Profiling Report - SNR Intervinientes sin duplicados clean")
dbutils.fs.put("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/intervinientes_sin_duplicados_clean.parquet_profile.html", profile.to_html() )

In [ ]:
df_clean.createOrReplaceTempView("intervinientes_clean")

In [ ]:
df_validar = spark.sql("""
select TIPO_DOCUMENTO, NUMERO_DOCUMENTO_CLEAN_LEN, count(id) as conteo
from intervinientes_clean  where TIPO = "NACIONAL"
group by TIPO_DOCUMENTO, NUMERO_DOCUMENTO_CLEAN_LEN
order by TIPO_DOCUMENTO, NUMERO_DOCUMENTO_CLEAN_LEN
""")
#df_validar.show()
df_validar.display()

In [ ]:
df_validar = spark.sql("""
select TIPO_DOCUMENTO, PERSONA_CLEAN, NUMERO_DOCUMENTO_CLEAN,
NUMERO_DOCUMENTO_CLEAN_LEN,
persona_num_words,
persona_num_commas
from intervinientes_clean
where TIPO_DOCUMENTO = 'P' and  NUMERO_DOCUMENTO_CLEAN_LEN > -1
order by NUMERO_DOCUMENTO_CLEAN_LEN asc, NUMERO_DOCUMENTO_CLEAN asc 
""")
#df_validar.show()
df_validar.display()

In [ ]:
df_clasificar_sexo = spark.sql("""
select id , PERSONA_CLEAN, NUMERO_DOCUMENTO_CLEAN
from intervinientes_clean
where TIPO_DOCUMENTO = 'C' and  NUMERO_DOCUMENTO_CLEAN_LEN < 9
""")
#df_validar.show()
df_clasificar_sexo = df_clasificar_sexo.withColumn("NUMERO_DOCUMENTO_CLEAN_INTEGER", df_clasificar_sexo.NUMERO_DOCUMENTO_CLEAN.cast('INTEGER'))
df_clasificar_sexo.display()

In [ ]:
df_clasificar_sexo.printSchema()

In [ ]:
# MASCULINO hombres eran del 001 hasta el 19.999.999, después solicitaron un  nuevo cupo que inicio en el  70.000.001 al 99.999.999.  
# FEMENINO 20000001 hasta el 69999999, 

def classify_sex(num_documento):
  if ( ( num_documento >= 20000000) &  ( num_documento <=  69999999 ) ):
    return "FEMENINO"
  else:
    return "MASCULINO"

classify_sex_udf = udf(classify_sex, StringType() )

#  Notas: hay algunas personas que separan apellido y nombre usando una coma 
df_clasificar_sexo = df_clasificar_sexo.withColumn("SEXO", classify_sex_udf(df_clasificar_sexo.NUMERO_DOCUMENTO_CLEAN_INTEGER)  )
df_clasificar_sexo.display()

In [ ]:
df_clasificar_sexo_for_join =  df_clasificar_sexo.select("id","SEXO")
df_clasificar_sexo_for_join = df_clasificar_sexo_for_join.withColumnRenamed("id", "fk")
df_clasificar_sexo_for_join.show()

In [ ]:
df_clean_join_sex = df_clean.join(df_clasificar_sexo_for_join, df_clean.id == df_clasificar_sexo_for_join.fk,   how='left') 
df_clean_join_sex = df_clean_join_sex.drop("RANGOS_CEDULA_SEXO")
df_clean_join_sex = df_clean_join_sex.drop("fk")
display(df_clean_join_sex) 

In [ ]:
df_clean_join_sex.printSchema()

In [ ]:
# filtrar empresas
# filtrar los que tienen TIPO_DOCUMENTO = 'C' and  NUMERO_DOCUMENTO_CLEAN_LEN = 9  y son empresas (palabras clave)
# los nombres con muchas palabras (ejm. +20 ) son de empresas o copy paste de otros campos
# A FAVOR SUYO SU CONYUGE O COMPAERO A PERMANENTE SUS HIJOS MENORES ACTUALES Y DE QUE LLEGARE A TENER TAMAYO
# los  TIPO_DOCUMENTO = ''  pueden tener personas o empresas.  no tienen número de documento



# clasificar por sexo los que no se pueden por rango:
# no tener en cuenta los tipo "E", 'P' (Extranjería o pasaporte)   EXTRAnJERO
# no tener en cuenta 'N' NIT ,   para la clasificación de sexo   Empresas
# si incluir para clasificar por sexo 'R', 'C' , 'T'


In [ ]:
#df_palabras_comunes_empresas == df_pce
df_pce = spark.read.json("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/DATOS_GOV_CO/nit_palabras_comunes_filtradas.json")
df_pce = df_pce.select("word")
display(df_pce)

In [ ]:
df_clean_join_sex.createOrReplaceTempView("intervinientes_sex")
df_pce.createOrReplaceTempView("pce")

In [ ]:
df_empresas_inferidas =   spark.sql("""
with words as ( 
SELECT id,  explode(  split(PERSONA_CLEAN, ' ') )  as word
FROM intervinientes_sex
where TIPO = 'NACIONAL' and TIPO_DOCUMENTO <> 'N'  )
select distinct w.id as id_empresa
from words as w inner join pce p on p.word = w.word
""")
display(df_empresas_inferidas)

In [ ]:
df_empresas_inferidas.count() # 729177  736787  738807

In [ ]:
df_clean_join_sex_empresas = df_clean_join_sex.join(df_empresas_inferidas, df_clean_join_sex.id == df_empresas_inferidas.id_empresa,   how='left') 
#df_clean_join_sex = df_clean_join_sex.drop("RANGOS_CEDULA_SEXO")
#df_clean_join_sex = df_clean_join_sex.drop("fk")
display(df_clean_join_sex_empresas)

In [ ]:
def classify_tipo_interviniente(tipo, tipo_documento, id_empresa ):
  if ( tipo == "NACIONAL" ):
    if (tipo_documento == "N" ):
      return "EMPRESA"
    if id_empresa is  None :
      return "PERSONA"
    else:
      return "EMPRESA"
  else:
    return ""

classify_tipo_interviniente_udf = udf(classify_tipo_interviniente, StringType() )

#  Notas: hay algunas personas que separan apellido y nombre usando una coma 
df_clean_join_sex_empresas = df_clean_join_sex_empresas.withColumn("TIPO_INTERVINENTE",
                           classify_tipo_interviniente_udf(df_clean_join_sex_empresas.TIPO, 
                                                           df_clean_join_sex_empresas.TIPO_DOCUMENTO  , 
                                                           df_clean_join_sex_empresas.id_empresa  )  )
df_clean_join_sex_empresas.display()

In [ ]:
df_clean_join_sex_empresas.createOrReplaceTempView("intervinientes_sex_empresas")

df_personas_procesar  = spark.sql("""
SELECT id, PERSONA_CLEAN, persona_num_words, persona_num_commas
FROM intervinientes_sex_empresas
where TIPO_INTERVINENTE = 'PERSONA' 
""")

df_personas_procesar.count()  # 4221483  4219463


In [ ]:
df_personas_procesar.show(truncate=False) 

In [ ]:
df_personas_procesar.cache()

In [ ]:
df_personas_procesar.createOrReplaceTempView("personas_procesar")



In [ ]:
df_apellido_nombre =  spark.sql("""
SELECT * 
FROM personas_procesar
where  persona_num_words = 2
""")

split_col = F.split(df_apellido_nombre['PERSONA_CLEAN'], ' ')
df_apellido_nombre = df_apellido_nombre.withColumn('APELLIDOS', split_col.getItem(0))
df_apellido_nombre = df_apellido_nombre.withColumn('NOMBRES', split_col.getItem(1))
df_apellido_nombre.show(n= 10, truncate=False) 


In [ ]:
df_apellido_multi_nombre =  spark.sql("""
SELECT * 
FROM personas_procesar
where  persona_num_words > 2 and persona_num_commas = 1
""")

split_col = F.split(df_apellido_multi_nombre['PERSONA_CLEAN'], '^(?:\S+\s){1}(\S+)'   )
df_apellido_multi_nombre = df_apellido_multi_nombre.withColumn("APELLIDOS", F.regexp_extract(df_apellido_multi_nombre.PERSONA_CLEAN, "([\w]+\s){2}", 0))
df_apellido_multi_nombre = df_apellido_multi_nombre.withColumn('NOMBRES', split_col.getItem(1)  )
df_apellido_multi_nombre.show(n= 10, truncate=False) 


In [ ]:
# se ignoran los nombres con más de 3 ","

df_apellido_sep_nombre =   spark.sql("""
SELECT *
FROM personas_procesar
where  persona_num_commas = 2
""")
split_col = F.split(df_apellido_sep_nombre['PERSONA_CLEAN'], ',')
df_apellido_sep_nombre = df_apellido_sep_nombre.withColumn('APELLIDOS', split_col.getItem(0))
df_apellido_sep_nombre = df_apellido_sep_nombre.withColumn('NOMBRES', split_col.getItem(1))
df_apellido_sep_nombre.show(n= 10, truncate=False) 

In [ ]:
df_all_nombres = df_apellido_nombre.union(df_apellido_multi_nombre).union(df_apellido_sep_nombre)
#df_all_nombres.count() # 4286067
df_all_nombres.show(n= 10, truncate=False) 

In [ ]:
df_all_nombres_for_join = df_all_nombres.select("id", "APELLIDOS", "NOMBRES" )
df_all_nombres_for_join = df_all_nombres_for_join.withColumn('APELLIDOS', F.trim(  df_all_nombres_for_join.APELLIDOS  )   )
df_all_nombres_for_join = df_all_nombres_for_join.withColumn('NOMBRES', F.trim(  df_all_nombres_for_join.NOMBRES  )   )
df_all_nombres_for_join = df_all_nombres_for_join.withColumn("PRIMER_NOMBRE", F.regexp_extract(df_all_nombres_for_join.NOMBRES, "(\w+)", 0))
df_all_nombres_for_join = df_all_nombres_for_join.withColumn("PRIMER_NOMBRE_ULT_LETR", F.regexp_extract(df_all_nombres_for_join.PRIMER_NOMBRE, "\w{1}$", 0))
df_all_nombres_for_join = df_all_nombres_for_join.withColumnRenamed("id", "fk")
display(df_all_nombres_for_join)

In [ ]:
df_clean_join_sex_empresas_nombres = df_clean_join_sex_empresas.join(df_all_nombres_for_join, 
                                                                     df_clean_join_sex_empresas.id == df_all_nombres_for_join.fk,   how='left') 

df_clean_join_sex_empresas_nombres = df_clean_join_sex_empresas_nombres.drop("fk")
display(df_clean_join_sex_empresas_nombres) 

In [ ]:
df_clean_join_sex_empresas_nombres.write.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/intervinientes_sexo_nombres.parquet")
profile = ProfileReport(df_clean_join_sex_empresas_nombres.select("*").toPandas(), title="Pandas Profiling Report - SNR Intervinientes Clasificados por Sexo")
dbutils.fs.put("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/intervinientes_sexo_nombres.parquet_profile.html", profile.to_html() )

In [ ]:
df_interv = spark.read.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/intervinientes_sexo_nombres.parquet")
df_interv.printSchema()

In [ ]:
df_interv.createOrReplaceTempView("intervinientes")

In [ ]:
df_export = spark.sql("""
with t as (
  select count(*) as total from intervinientes where TIPO_INTERVINENTE = 'PERSONA' and SEXO = 'MASCULINO' 
), f as ( 
  SELECT SEXO, PRIMER_NOMBRE, count(*) AS frecuencia 
  FROM intervinientes as i  
  where i.TIPO_INTERVINENTE = 'PERSONA' and  i.SEXO = 'MASCULINO'
  group by SEXO, PRIMER_NOMBRE 
)
select  f.SEXO, f.PRIMER_NOMBRE, f.frecuencia, f.frecuencia / t.total as prob
from f , t
order by f.SEXO ASC , f.frecuencia DESC 



""")
display(df_export) 

In [ ]:
df_export2 = spark.sql("""
with t as (
  select count(*) as total from intervinientes where TIPO_INTERVINENTE = 'PERSONA' and SEXO = 'FEMENINO' 
), f as ( 
  SELECT SEXO, PRIMER_NOMBRE, count(*) AS frecuencia 
  FROM intervinientes as i  
  where i.TIPO_INTERVINENTE = 'PERSONA' and  i.SEXO = 'FEMENINO'
  group by SEXO, PRIMER_NOMBRE 
)
select  f.SEXO, f.PRIMER_NOMBRE, f.frecuencia, f.frecuencia / t.total as prob
from f , t
order by f.SEXO ASC , f.frecuencia DESC 



""")
display(df_export2) 

In [ ]:
df_export_all = df_export.union(df_export2)
df_export_all.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/sexo_nombre_prob.json")

In [ ]:
df_export = spark.sql("""
SELECT PRIMER_NOMBRE, count(*) AS TOTAL 
FROM intervinientes  
where TIPO_INTERVINENTE = 'PERSONA' and SEXO IS NULL
group by PRIMER_NOMBRE 
order by  count(*) DESC , PRIMER_NOMBRE ASC 
""")
display(df_export) 

In [ ]:
df_export.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/nombres_para_clasificar.json")

In [ ]:
df_export = spark.sql("""
with h as ( 
  SELECT PRIMER_NOMBRE, count(*) AS TOTAL 
  FROM intervinientes  
  where TIPO_INTERVINENTE = 'PERSONA' and SEXO = 'MASCULINO'
  group by PRIMER_NOMBRE
  ) ,  f as (
  SELECT PRIMER_NOMBRE, count(*) AS TOTAL 
  FROM intervinientes  
  where TIPO_INTERVINENTE = 'PERSONA' and SEXO = 'FEMENINO'
  group by PRIMER_NOMBRE
)
select h.PRIMER_NOMBRE, h.TOTAL AS TOTAL_HOMBRES, f.TOTAL as TOTAL_MUJERES
from h inner join f on h.PRIMER_NOMBRE = f.PRIMER_NOMBRE
ORDER BY h.TOTAL DESC, f.TOTAL desc 
""")
display(df_export) 

In [ ]:
df_export.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/nombres_comunes_masculino_femenino.json")

In [ ]:
df_export = spark.sql("""

SELECT SEXO, PRIMER_NOMBRE_ULT_LETR, count(*) AS TOTAL 
FROM intervinientes  
where TIPO_INTERVINENTE = 'PERSONA' and SEXO IS NOT NULL
group by SEXO, PRIMER_NOMBRE_ULT_LETR
ORDER BY SEXO , COUNT(*) DESC
  """)

display(df_export) 

In [ ]:
df_export.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/sexo_ultima_letra.json")